In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [2]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [77]:
AWS_folder = os.getcwd()

b_folder = os.path.join(AWS_folder,'BACKUPS')
b_folder

src_folder = os.path.join(AWS_folder,'dataDownloaded')
src_folder

out_folder = 'c:\\Users\\yc463\\Documents'
out_folder

'c:\\Users\\yc463\\Documents'

In [78]:
# alldata[0]

In [79]:
recentDB = glob.glob(os.path.join(b_folder ,'Total*.pickle'), recursive=True)[-1]
df = pd.read_pickle(recentDB)

In [80]:
df['originalFile'] = df.json.map(lambda x: x['originalFileJPG'])
df['label'] = df.json.map(lambda x: x['label'])

In [81]:

def extract_root(x):
  l = text_.split('/')
  index_ = [idx for idx, s in enumerate(l) if '2023' and '완' in s][0]
  return ('/').join(l[index_:])

df['originalFile'] = df['originalFile'].map(lambda x: extract_root(x))

In [82]:
def get_simLabel(x):
  return [(label_.get('level1','') or '')+ '_' + (label_.get('level2','') or '') for label_ in x if len(x)!=0]

df['label'] = df['label'].map(lambda x: get_simLabel(x))

In [89]:
report_df = pd.concat([df[['courseid','date','area','desc','originalFile','id']], pd.DataFrame(df['label'].values.tolist())], axis=1).fillna('')

In [90]:
report_df

,courseid,date,area,desc,originalFile,id,0,1,2,3,4,5
0,MGC004,20230919,7H,"브라운패취(그린),조류(그린),볼마크(그린)_0",2023_0919_시흥 아세코밸리_데이터 1차 분류_완/7H/2023_0919_시흥...,20230919114349_1267681724167_373693833056_4798...,병해_패취,,,,,
1,MGC004,20230919,7H,"브라운패취(그린),조류(그린),볼마크(그린)_0",2023_0919_시흥 아세코밸리_데이터 1차 분류_완/7H/2023_0919_시흥...,20230919114350_1267681720833_373693834444_4797...,병해_패취,,,,,
2,MGC004,20230919,7H,"브라운패취(그린),조류(그린),볼마크(그린)_0",2023_0919_시흥 아세코밸리_데이터 1차 분류_완/7H/2023_0919_시흥...,20230919114352_1267681734167_373693834722_4797...,병해_패취,,,,,
3,MGC004,20230919,7H,"브라운패취(그린),조류(그린),볼마크(그린)_0",2023_0919_시흥 아세코밸리_데이터 1차 분류_완/7H/2023_0919_시흥...,20230919114354_1267681742778_373693835556_4797...,병해_패취,,,,,
4,MGC004,20230919,7H,"브라운패취(그린),조류(그린),볼마크(그린)_0",2023_0919_시흥 아세코밸리_데이터 1차 분류_완/7H/2023_0919_시흥...,20230919114356_1267681752500_373693833056_4797...,병해_패취,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
176300,MGC001,20230816,6.태백1H,미기재_0,2023_0919_시흥 아세코밸리_데이터 1차 분류_완/7H/2023_0919_시흥...,20230816160444_1293353953333_362254629444_1552...,,,,,,
176301,MGC001,20230816,6.태백1H,미기재_0,2023_0919_시흥 아세코밸리_데이터 1차 분류_완/7H/2023_0919_시흥...,20230816160446_1293353841944_362254667778_1552...,,,,,,
176302,MGC001,20230816,6.태백1H,미기재_0,2023_0919_시흥 아세코밸리_데이터 1차 분류_완/7H/2023_0919_시흥...,20230816160448_1293353721667_362254712500_1552...,,,,,,
176303,MGC001,20230816,6.태백1H,미기재_0,2023_0919_시흥 아세코밸리_데이터 1차 분류_완/7H/2023_0919_시흥...,20230816160451_1293353597778_362254760000_1552...,,,,,,


In [91]:
sum_df = report_df.melt(id_vars=['courseid','date','area','desc','originalFile','id'], 
        var_name="Label#", 
        value_name="Label")

In [92]:
sum_df.groupby(['courseid','date','Label']).agg({'id':'count'}).to_excel(os.path.join(out_folder,'NIA_DS_Report_{}.xlsx'.format(datetime.now().strftime("%Y%m%d_%H%M%S"))))

In [96]:
df.groupby(['courseid','date']).agg({'id':'count'}).to_excel(os.path.join(out_folder,'NIA_DS_Report_{}.xlsx'.format(datetime.now().strftime("%Y%m%d"))))